In [1]:
# https://github.com/ultralytics/ultralytics/issues/1429#issuecomment-1519239409

from pathlib import Path
import torch
import argparse
import numpy as np
import cv2
from types import SimpleNamespace

from boxmot.tracker_zoo import create_tracker
from boxmot.utils import ROOT, WEIGHTS
from boxmot.utils.checks import TestRequirements
from boxmot.utils import logger as LOGGER
from boxmot.utils.torch_utils import select_device

tr = TestRequirements()
tr.check_packages(('ultralytics',))  # install
%load_ext autoreload
%autoreload 2
from ultralytics.yolo.engine.model import YOLO, TASK_MAP

from ultralytics.yolo.utils import SETTINGS, colorstr, ops, is_git_dir, IterableSimpleNamespace
from ultralytics.yolo.utils.checks import check_imgsz, print_args
from ultralytics.yolo.utils.files import increment_path
from ultralytics.yolo.engine.results import Boxes
from ultralytics.yolo.data.utils import VID_FORMATS
from ultralytics.yolo.utils.plotting import save_one_box

from multi_yolo_backend import MultiYolo
from utils import write_MOT_results


def on_predict_start(predictor):
    predictor.trackers = []
    predictor.tracker_outputs = [None] * predictor.dataset.bs
    predictor.args.tracking_config = \
        ROOT /\
        'boxmot' /\
        opt.tracking_method /\
        'configs' /\
        (opt.tracking_method + '.yaml')
    for i in range(predictor.dataset.bs):
        tracker = create_tracker(
            predictor.args.tracking_method,
            predictor.args.tracking_config,
            predictor.args.reid_model,
            predictor.device,
            predictor.args.half
        )
        predictor.trackers.append(tracker)




def parse_opt(yolo_model=WEIGHTS / 'yolov8n.pt', reid_model=WEIGHTS / 'mobilenetv2_x1_4_dukemtmcreid.pt',
              tracking_method='deepocsort', source='aiworkspace.jpg', imgsz=[640], conf=0.5, iou=0.7, device='', show=False,
              save=False, classes=None, project=ROOT / 'runs' / 'track', name='exp', exist_ok=False, half=False,
              vid_stride=1, hide_label=False, hide_conf=False, save_txt=False, save_id_crops=False, save_mot=False):
    opt = argparse.Namespace(
        yolo_model=yolo_model,
        reid_model=reid_model,
        tracking_method=tracking_method,
        source=source,
        imgsz=imgsz,
        conf=conf,
        iou=iou,
        device=device,
        show=show,
        save=save,
        classes=classes,
        project=project,
        name=name,
        exist_ok=exist_ok,
        half=half,
        vid_stride=vid_stride,
        hide_label=hide_label,
        hide_conf=hide_conf,
        save_txt=save_txt,
        save_id_crops=save_id_crops,
        save_mot=save_mot
    )
    return opt



In [2]:

opt = parse_opt()

args = vars(opt)

In [14]:
model = YOLO(args['yolo_model'] if 'v8' in str(
    args['yolo_model']) else 'yolov8n')
overrides = model.overrides.copy()
model.predictor = TASK_MAP[model.task][3](
    overrides=overrides, _callbacks=model.callbacks)

# extract task predictor
predictor = model.predictor

# combine default predictor args with custom, preferring custom
combined_args = {**predictor.args.__dict__, **args}
# overwrite default args
predictor.args = IterableSimpleNamespace(**combined_args)
predictor.args.device = select_device(args['device'])


# setup source and model
if not predictor.model:
    predictor.setup_model(model=model.model, verbose=False)
predictor.setup_source(predictor.args.source)

predictor.args.imgsz = check_imgsz(
    predictor.args.imgsz, stride=model.model.stride, min_dim=2)  # check image size
predictor.save_dir = increment_path(Path(
    predictor.args.project) / predictor.args.name, exist_ok=predictor.args.exist_ok)

# Check if save_dir/ label file exists
if predictor.args.save or predictor.args.save_txt:
    (predictor.save_dir / 'labels' if predictor.args.save_txt else predictor.save_dir).mkdir(parents=True, exist_ok=True)
# Warmup model
if not predictor.done_warmup:
    predictor.model.warmup(imgsz=(
        1 if predictor.model.pt or predictor.model.triton else predictor.dataset.bs, 3, *predictor.imgsz))
    predictor.done_warmup = True
predictor.seen, predictor.windows, predictor.batch, predictor.profilers = 0, [
], None, (ops.Profile(), ops.Profile(), ops.Profile(), ops.Profile())
predictor.add_callback('on_predict_start', on_predict_start)
predictor.run_callbacks('on_predict_start')
model = MultiYolo(
    model=model.predictor.model if 'v8' in str(
        args['yolo_model']) else args['yolo_model'],
    device=predictor.device,
    args=predictor.args
)
for frame_idx, batch in enumerate(predictor.dataset):
    predictor.run_callbacks('on_predict_batch_start')
    predictor.batch = batch
    path, im0s, vid_cap, s = batch
    visualize = increment_path(save_dir / Path(path[0]).stem, exist_ok=True, mkdir=True) if predictor.args.visualize and (
        not predictor.dataset.source_type.tensor) else False

    n = len(im0s)
    predictor.results = [None] * n

    # Preprocess
    with predictor.profilers[0]:
        im = predictor.preprocess(im0s)

    # Inference
    with predictor.profilers[1]:
        preds = model(im, im0s)

2023-06-30 19:54:38.685 | INFO     | boxmot.utils.torch_utils:select_device:52 - Yolo Tracking v10.0.15 🚀 Python-3.10.6 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3090, 24226MiB)



#, m.type, x.shape 0 ultralytics.nn.modules.Conv torch.Size([1, 3, 640, 640])
#, m.type, x.shape 1 ultralytics.nn.modules.Conv torch.Size([1, 16, 320, 320])
#, m.type, x.shape 2 ultralytics.nn.modules.C2f torch.Size([1, 32, 160, 160])
#, m.type, x.shape 3 ultralytics.nn.modules.Conv torch.Size([1, 32, 160, 160])
#, m.type, x.shape 4 ultralytics.nn.modules.C2f torch.Size([1, 64, 80, 80])
#, m.type, x.shape 5 ultralytics.nn.modules.Conv torch.Size([1, 64, 80, 80])
#, m.type, x.shape 6 ultralytics.nn.modules.C2f torch.Size([1, 128, 40, 40])
#, m.type, x.shape 7 ultralytics.nn.modules.Conv torch.Size([1, 128, 40, 40])
#, m.type, x.shape 8 ultralytics.nn.modules.C2f torch.Size([1, 256, 20, 20])
#, m.type, x.shape 9 ultralytics.nn.modules.SPPF torch.Size([1, 256, 20, 20])
#, m.type, x.shape 10 torch.nn.modules.upsampling.Upsample torch.Size([1, 256, 20, 20])
#, m.type, len(x) 11 ultralytics.nn.modules.Concat 2
#, m.type, x.shape 12 ultralytics.nn.modules.C2f torch.Size([1, 384, 40, 40])
#, m

2023-06-30 19:54:46.882 | SUCCESS  | boxmot.deep.reid_model_factory:load_pretrained_weights:229 - Successfully loaded pretrained weights from "/home/juma/code/luqmon_tracking/examples/weights/mobilenetv2_x1_4_dukemtmcreid.pt"
2023-06-30 19:54:46.883 | WARNING  | boxmot.deep.reid_model_factory:load_pretrained_weights:233 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')


#, m.type, x.shape 0 ultralytics.nn.modules.Conv torch.Size([1, 3, 320, 640])
#, m.type, x.shape 1 ultralytics.nn.modules.Conv torch.Size([1, 16, 160, 320])
#, m.type, x.shape 2 ultralytics.nn.modules.C2f torch.Size([1, 32, 80, 160])
#, m.type, x.shape 3 ultralytics.nn.modules.Conv torch.Size([1, 32, 80, 160])
#, m.type, x.shape 4 ultralytics.nn.modules.C2f torch.Size([1, 64, 40, 80])
#, m.type, x.shape 5 ultralytics.nn.modules.Conv torch.Size([1, 64, 40, 80])
#, m.type, x.shape 6 ultralytics.nn.modules.C2f torch.Size([1, 128, 20, 40])
#, m.type, x.shape 7 ultralytics.nn.modules.Conv torch.Size([1, 128, 20, 40])
#, m.type, x.shape 8 ultralytics.nn.modules.C2f torch.Size([1, 256, 10, 20])
#, m.type, x.shape 9 ultralytics.nn.modules.SPPF torch.Size([1, 256, 10, 20])
#, m.type, x.shape 10 torch.nn.modules.upsampling.Upsample torch.Size([1, 256, 10, 20])
#, m.type, len(x) 11 ultralytics.nn.modules.Concat 2
#, m.type, x.shape 12 ultralytics.nn.modules.C2f torch.Size([1, 384, 20, 40])
#, m.t

In [ ]:
model.model

AutoBackend(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): ModuleList(
          (0): Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): Conv(
              (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=Tr

In [8]:
preds.shape

AttributeError: 'list' object has no attribute 'shape'

In [9]:
len(preds)

2

In [ ]:
preds[0].shape

torch.Size([1, 84, 4200])

In [ ]:
len(preds[1])

3

In [ ]:
preds[1][0].shape

torch.Size([1, 144, 40, 80])

In [ ]:
preds[1][1].shape

torch.Size([1, 144, 20, 40])

In [ ]:
preds[1][2].shape

torch.Size([1, 144, 10, 20])